pip install transformers[torch] datasets evaluate opendatasets

In [1]:
import os, mlflow
import datasets
import transformers
import evaluate
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
import opendatasets as od
import pandas as pd
import numpy as np
import pickle as pkl
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from matplotlib import pyplot as plt
import torch

c:\Users\Abelda-san\anaconda3\envs\factcheck\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["MLFLOW_EXPERIMENT_NAME"] = "factcheck"
os.environ["MLFLOW_FLATTEN_PARAMS"] = "1"
os.environ["MLFLOW_TRACKING_URI"]="http://127.0.0.1:5000"
os.environ["HF_MLFLOW_LOG_ARTIFACTS"]="1"

In [3]:
od.download(
    "https://www.kaggle.com/datasets/rmisra/politifact-fact-check-dataset/data"
    )
data = datasets.Dataset.from_json("politifact-fact-check-dataset/politifact_factcheck_data.json")
data = data.train_test_split(train_size=0.8)

Using custom data configuration default-58a22729889bbb0b


Skipping, found downloaded files in ".\politifact-fact-check-dataset" (use force=True to force download)


Found cached dataset json (C:/Users/Abelda-san/.cache/huggingface/datasets/json/default-58a22729889bbb0b/0.0.0)


In [4]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

label2id = {
    'true': 0,
    'mostly-true': 1,
    'half-true': 2,
    'mostly-false': 3,
    'false': 4,
    'pants-fire': 5
}
id2label = {
    0: 'true',
    1: 'mostly-true',
    2: 'half-true',
    3: 'mostly-false',
    4: 'false',
    5: 'pants-fire'
}
num_labels = len(label2id)

def preprocess_data(examples):

    text = examples["statement"]
    labels = pd.Series(examples['verdict'])

    encoding = tokenizer(text, padding=True, truncation=True)
    encoding['labels'] = list(labels.apply(lambda x: [1.0 if label2id[x]==i else 0.0 for i in range(num_labels)]))

    return encoding


train_tokenized_dataset = data["train"].map(preprocess_data, batched=True, remove_columns=data["train"].column_names)
test_tokenized_dataset = data["test"].map(preprocess_data, batched=True, remove_columns=data["test"].column_names)

train_tokenized_dataset.set_format("torch")
test_tokenized_dataset.set_format("torch")

 80%|████████  | 4/5 [00:00<00:00,  7.72ba/s]


In [5]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                            problem_type="multi_label_classification", 
                                                            num_labels=6,
                                                            id2label=id2label,
                                                            label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [6]:
training_args = TrainingArguments(
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=15,
    per_device_eval_batch_size=15,
    num_train_epochs=5,
    load_best_model_at_end=True,
    metric_for_best_model='f1'
)

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [8]:
trainer.train()

***** Running training *****
  Num examples = 16921
  Num Epochs = 5
  Instantaneous batch size per device = 15
  Total train batch size (w. parallel, distributed & accumulation) = 15
  Gradient Accumulation steps = 1
  Total optimization steps = 5645
  1%|          | 33/5645 [03:10<8:58:55,  5.76s/it]


In [ ]:
mlflow.end_run()